In [160]:
import pandas as pd
import workdays
import datetime
from datetime import datetime, timedelta, date
from xbbg import blp
import pdblp
# https://data.bloomberglp.com/labs/sites/2/2013/12/blpapi-developers-guide-1.38.pdf
# https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf

In [161]:
FX0 = ['USD', 'GBP', 'HKD', 'CAD', 'AUD', 'EUR','CHF']
FX = [x + 'JPY BGN Curncy' for x in FX0 ]
FX = blp.bdp(tickers=FX, flds='px_last')
FX.index = [x[:3] for x in FX.index.tolist()]
FX

,px_last
AUD,94.948
CAD,105.370
CHF,170.314
EUR,162.110
GBP,194.421
HKD,19.364
USD,150.590


In [162]:
def BDP(L,YK):
    T = [l + 'A ' + YK for l in L ]
    T1 = [l + '1 ' + YK for l in L ]

    df = blp.bdp(tickers=T, flds=['name','currency','exch_code','undl_spot_ticker','fut_trading_hrs']).loc[T,:]
    tmp = [x + ' Index' for x in df['undl_spot_ticker'].drop_duplicates().tolist()]
    tmp = blp.bdp(tickers=tmp, flds='country_iso')
    tmp.index = [item.replace(' Index', '') for item in tmp.index.tolist()]

    df.loc[:, 'risk'] = df['undl_spot_ticker'].map(tmp['country_iso'])
    df.insert(0,'ticker', L)
    df = df.reset_index(drop=True)

    df1 = blp.bdp(tickers=T1, flds=['contract_value','volume_avg_20d','volatility_20d']).loc[T1,:]
    df1.columns = ['value/unit','volume','vola']
    df = pd.concat([df, df1.reset_index(drop=True)], axis=1)
    df = df.rename(columns={'currency':'crncy','exch_code':'exch','undl_spot_ticker':'undl','contract_value':'value/unit'})
    
    tmp = []
    for i in range(len(L)):
        tmp  += [(blp.bdh(T1[i], 'px_last', '19991201', '20220101', Per='Q' ).reset_index().iloc[0,0]).strftime('%Y%m')]
    df['initinal_t/d'] = tmp
    df = df.replace('199912', '-')

    df['initinal_t/d'] = 0
    
    df['fx'] = 1.0
    df.loc[df['crncy'] != 'JPY', 'fx'] = df['crncy'].map(FX['px_last'])
    df['value/unit_JPY'] = (df['value/unit'] * df['fx']).round(-3)
    df = df.drop('fx', axis=1)

    tmp = df['risk']
    df = df.drop('risk', axis=1)
    df.insert(3, 'risk', tmp)

    df = df.drop('value/unit', axis=1)
    df['value/unit_JPY'] = df.pop('value/unit_JPY')
    df['initinal_t/d'] = df.pop('initinal_t/d')


    df['vola'] = df['vola'].round()
    df['volume'] = df['volume'].fillna(0)
    
    df['illiquidity'] = 0
    df['mini'] = '-'
    df['size_mini'] = '-'
    df['illiquidity_mini'] = 0
    
    for i in range(len(df)):
        
        if df.loc[i,'volume'] < 3000:
            df.loc[i,'illiquidity'] = 1
        
        if df.loc[:i,'undl'].tolist().count(df.loc[i,'undl']) > 1:
            df.loc[i,'mini'] = 1
            
            r = df.loc[df['undl'] == df.loc[i,'undl'],'size_mini'].index[0]
            df.loc[r,'mini'] = df.loc[i,'ticker']
            df.loc[r,'size_mini'] = (df.loc[i,'value/unit_JPY'] / df.loc[r,'value/unit_JPY']).round(3)
            
            if df.loc[i,'illiquidity'] == 1:
                df.loc[r,'illiquidity_mini'] = 1
            
    for j in ['value/unit_JPY','volume']:
        df[j] = df[j].apply('{:,.0f}'.format)


    for i in range(len(df)):
        df.loc[i,'name'] = df.loc[i,'name'][:df.loc[i,'name'].rfind(' ')]

    for j in [' FUT',' Future',' Index','  INDXR', ' IDX', ' IXR']:
        df.loc[i,'name'] = df.loc[i,'name'].replace(j, '')

    return df
    
L= ['ES','NQ','DM','RTY','VG','CF','GX','EO','IB','SW','Z ','SM','PT','XP','TP','HI','XU','JGS','TWT']
L += ['HWA','HWB','HWI','HWR','DFW','CMS','MFO','ID','MPT','MXP','TMI','HU','MES']

YK = ['Index'] * len(L)
YK = 'Index'
df = BDP(L,YK)


for i in range(len(df)):
    df.loc[i,'name'] = df.loc[i,'name'][:df.loc[i,'name'].rfind(' ')]

    for j in [' FUT',' Future',' Index','  INDXR', ' IDX', ' IXR']:
        df.loc[i,'name'] = df.loc[i,'name'].replace(j, '')


print('Equity Futures')
print(df.shape)
display(df[df['mini'] != 1])

print('mini(smaller size)')
display(df[df['mini'] == 1])
print('illiquidity')
df[df['illiquidity'] == 1]

Equity Futures
(32, 15)


,ticker,name,crncy,risk,exch,undl,fut_trading_hrs,volume,vola,value/unit_JPY,initinal_t/d,illiquidity,mini,size_mini,illiquidity_mini
0,ES,S&P500 EMINI,USD,US,CME,SPX,07:00-06:00,"2,145,951",21.0,"43,349,000",0,0,HWA,0.1,0
1,NQ,NASDAQ 100,USD,US,CME,NDX,07:00-06:00,"707,828",28.0,"60,479,000",0,0,HWB,0.1,0
2,DM,DJIA MINI e-CBOT,USD,US,CBT,INDU,07:00-06:00,"149,217",17.0,"32,226,000",0,0,HWI,0.2,0
3,RTY,E-Mini Russ 2000,USD,US,CME,RTY,07:00-06:00,"246,433",24.0,"15,763,000",0,0,HWR,0.1,0
4,VG,EURO STOXX 50,EUR,EU,EUX,SX5E,09:10-06:00,"909,332",22.0,"8,603,000",0,0,-,-,0
5,CF,CAC40 10 EURO,EUR,FR,EOP,CAC,16:00-06:00,"57,603",16.0,"12,912,000",0,0,CMS,0.1,1
6,GX,DAX INDEXURE,EUR,DE,EUX,DAX,09:10-06:00,"48,757",29.0,"92,427,000",0,0,DFW,0.2,0
7,EO,AMSTERDAM,EUR,NL,EOE,AEX,16:00-06:00,"10,591",13.0,"29,660,000",0,0,MFO,0.1,1
8,IB,IBEX 35 INDX,EUR,ES,MFM,IBEX,16:00-04:00,"14,425",19.0,"21,554,000",0,0,ID,0.1,0
9,SW,MINI FTSE/MIB,EUR,IT,MIL,FTSEMIB,16:00-06:00,"8,904",22.0,"6,148,000",0,0,-,-,0


mini(smaller size)


,ticker,name,crncy,risk,exch,undl,fut_trading_hrs,volume,vola,value/unit_JPY,initinal_t/d,illiquidity,mini,size_mini,illiquidity_mini
19,HWA,SP500 MIC,USD,US,CME,SPX,07:00-06:00,"1,594,431",21.0,"4,335,000",0,0,1,-,0
20,HWB,NASD100,USD,US,CME,NDX,07:00-06:00,"2,098,741",28.0,"6,048,000",0,0,1,-,0
21,HWI,DJIA MICR,USD,US,CBT,INDU,07:00-06:00,"158,250",17.0,"6,445,000",0,0,1,-,0
22,HWR,MICRO,USD,US,CME,RTY,07:00-06:00,"86,604",24.0,"1,577,000",0,0,1,-,0
23,DFW,DAX Mini,EUR,DE,EUX,DAX,09:10-06:00,"37,417",29.0,"18,487,000",0,0,1,-,0
24,CMS,CAC40 Mini,EUR,FR,EOP,CAC,16:00-06:00,147,16.0,"1,287,000",0,1,1,-,0
25,MFO,AMSTERDAM MIN ID,EUR,NL,EOE,AEX,16:00-06:00,8,13.0,"2,963,000",0,1,1,-,0
26,ID,IBEX MINI,EUR,ES,MFM,IBEX,16:00-04:00,"3,091",19.0,"2,154,000",0,0,1,-,0
27,MPT,MINI S&P/TSX 60,CAD,CA,MSE,SPTSX60,09:00-22:15 & 22:30-05:30,46,18.0,"7,961,000",0,1,1,-,0
28,MXP,Mini SPI 200,AUD,AU,SFE,AS51,15:10-05:00 & 07:50-14:30,0,13.0,"3,805,000",0,1,1,-,0


illiquidity


,ticker,name,crncy,risk,exch,undl,fut_trading_hrs,volume,vola,value/unit_JPY,initinal_t/d,illiquidity,mini,size_mini,illiquidity_mini
24,CMS,CAC40 Mini,EUR,FR,EOP,CAC,16:00-06:00,147,16.0,"1,287,000",0,1,1,-,0
25,MFO,AMSTERDAM MIN ID,EUR,NL,EOE,AEX,16:00-06:00,8,13.0,"2,963,000",0,1,1,-,0
27,MPT,MINI S&P/TSX 60,CAD,CA,MSE,SPTSX60,09:00-22:15 & 22:30-05:30,46,18.0,"7,961,000",0,1,1,-,0
28,MXP,Mini SPI 200,AUD,AU,SFE,AS51,15:10-05:00 & 07:50-14:30,0,13.0,"3,805,000",0,1,1,-,0


In [163]:
df1 = df.loc[df['mini']!=1,['ticker','name','crncy','value/unit_JPY','mini','size_mini','illiquidity_mini']].copy()
df1.loc[df1['illiquidity_mini']==1,'mini'] = '(illiquidity)'
df1.loc[df1['illiquidity_mini']==1,'size_mini'] = '-'
df1 = df1.drop('illiquidity_mini', axis=1)

for i in [200,300,500,1000]:
    df1[str(i) + ' mil'] = df1['value/unit_JPY'].str.replace(',', '').astype(float) * df1['size_mini'].replace('-', 1) / (i * 10**6)
    df1[str(i) + ' mil'] = df1[str(i) + ' mil'].apply('{:,.1%}'.format)

df1

C:\Users\A1800401\AppData\Local\Temp\ipykernel_61748\1180419246.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1[str(i) + ' mil'] = df1['value/unit_JPY'].str.replace(',', '').astype(float) * df1['size_mini'].replace('-', 1) / (i * 10**6)
C:\Users\A1800401\AppData\Local\Temp\ipykernel_61748\1180419246.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1[str(i) + ' mil'] = df1['value/unit_JPY'].str.replace(',', '').astype(float) * df1['size_mini'].replace('-', 1) / (i * 10**6)
C:\Users\A1800401\AppData\Local\Temp\ipykernel

,ticker,name,crncy,value/unit_JPY,mini,size_mini,200 mil,300 mil,500 mil,1000 mil
0,ES,S&P500 EMINI,USD,"43,349,000",HWA,0.1,2.2%,1.4%,0.9%,0.4%
1,NQ,NASDAQ 100,USD,"60,479,000",HWB,0.1,3.0%,2.0%,1.2%,0.6%
2,DM,DJIA MINI e-CBOT,USD,"32,226,000",HWI,0.2,3.2%,2.1%,1.3%,0.6%
3,RTY,E-Mini Russ 2000,USD,"15,763,000",HWR,0.1,0.8%,0.5%,0.3%,0.2%
4,VG,EURO STOXX 50,EUR,"8,603,000",-,-,4.3%,2.9%,1.7%,0.9%
5,CF,CAC40 10 EURO,EUR,"12,912,000",(illiquidity),-,6.5%,4.3%,2.6%,1.3%
6,GX,DAX INDEXURE,EUR,"92,427,000",DFW,0.2,9.2%,6.2%,3.7%,1.8%
7,EO,AMSTERDAM,EUR,"29,660,000",(illiquidity),-,14.8%,9.9%,5.9%,3.0%
8,IB,IBEX 35 INDX,EUR,"21,554,000",ID,0.1,1.1%,0.7%,0.4%,0.2%
9,SW,MINI FTSE/MIB,EUR,"6,148,000",-,-,3.1%,2.0%,1.2%,0.6%


In [171]:
def BDP(L,YK,Sort=0):
    T = [l + 'A ' + YK for l in L ]
    T1 = [l + '1 ' + YK for l in L ]

    df = blp.bdp(tickers=T, flds=['name','currency','exch_code','fut_trading_hrs']).loc[T,:]
    df.insert(0,'ticker', L)

    df1 = blp.bdp(tickers=T1, flds=['contract_value','volume_avg_20d']).loc[T1,:]
    df1.columns = ['value/unit','volume']
    df = pd.concat([df.reset_index(drop=True), df1.reset_index(drop=True)], axis=1)
    df = df.rename(columns={'currency':'crncy','exch_code':'exch','undl_spot_ticker':'undl','contract_value':'value/unit'})
    
    df['fx'] = 1.0
    df.loc[df['crncy'] != 'JPY', 'fx'] = df['crncy'].map(FX['px_last'])
    df['value/unit_JPY'] = (df['value/unit'] * df['fx']).round(-3)
    df = df.drop('fx', axis=1)

    df = df.drop('value/unit', axis=1)
    df['value/unit_JPY'] = df.pop('value/unit_JPY')

    df['volume'] = df['volume'].fillna(0)
    df['illiquidity'] = 0

    for i in range(len(df)):        
        if df.loc[i,'volume'] < 3000:
            df.loc[i,'illiquidity'] = 1
        
    for i in range(len(df)):
        df.loc[i,'name'] = df.loc[i,'name'][:df.loc[i,'name'].rfind(' ')]

    for j in [' FUT',' Future',' Index','  INDXR', ' IDX', ' IXR']:
        df.loc[i,'name'] = df.loc[i,'name'].replace(j, '')
    
    df1 = df.loc[:,['ticker','name','crncy','value/unit_JPY']].copy()
    
    
    if Sort ==1:
        df1 = df1.sort_values(by='value/unit_JPY').reset_index(drop=True)
    
    for i in [200,300,500,1000]:
        df1[str(i) + ' mil'] = df1['value/unit_JPY']  / (i * 10**6)
        df1[str(i) + ' mil'] = df1[str(i) + ' mil'].apply('{:,.1%}'.format)
    
    for j in ['value/unit_JPY']:
        df1[j] = df1[j].apply('{:,.0f}'.format)

    return df1

In [176]:
L = ['ES','NQ']
df = BDP(L,'Index',1)
df

,ticker,name,crncy,value/unit_JPY,200 mil,300 mil,500 mil,1000 mil
0,ES,S&P500 EMINI FUT,USD,"43,368,000",21.7%,14.5%,8.7%,4.3%
1,NQ,NASDAQ 100 E-MINI,USD,"60,505,000",30.3%,20.2%,12.1%,6.1%


In [177]:
L = ['CL','CO','NG','GC','MGC','HG','C ', 'S ','W ']
df = BDP(L,'Comdty',1)
df

,ticker,name,crncy,value/unit_JPY,200 mil,300 mil,500 mil,1000 mil
0,C,CORN FUTURE,USD,"3,392,000",1.7%,1.1%,0.7%,0.3%
1,W,WHEATURE(CBT),USD,"4,013,000",2.0%,1.3%,0.8%,0.4%
2,MGC,MICRO GOLD,USD,"4,585,000",2.3%,1.5%,0.9%,0.5%
3,NG,NATURAL GAS FUTR,USD,"5,628,000",2.8%,1.9%,1.1%,0.6%
4,S,SOYBEAN FUTURE,USD,"7,539,000",3.8%,2.5%,1.5%,0.8%
5,CL,WTI CRUDE FUTURE,USD,"10,451,000",5.2%,3.5%,2.1%,1.0%
6,CO,BRENT CRUDE FUTR,USD,"11,068,000",5.5%,3.7%,2.2%,1.1%
7,HG,COPPER FUTURE,USD,"19,703,000",9.9%,6.6%,3.9%,2.0%
8,GC,GOLD 100 OZ FUTR,USD,"46,293,000",23.1%,15.4%,9.3%,4.6%


In [173]:
L= ['TU','FV','TY','US','WN','RX','OAT','IK','Z ','JB','BJ']
df = BDP(L,'Comdty',1)
df

,ticker,name,crncy,value/unit_JPY,200 mil,300 mil,500 mil,1000 mil
0,BJ,10YR MINI JGB,JPY,"13,747,000",6.9%,4.6%,2.7%,1.4%
1,FV,US 5YR NOTE (CBT),USD,"16,207,000",8.1%,5.4%,3.2%,1.6%
2,TY,US 10YR NOTE,USD,"16,626,000",8.3%,5.5%,3.3%,1.7%
3,Z,FTSE 100 IDX FUT,GBP,"16,823,000",8.4%,5.6%,3.4%,1.7%
4,US,US LONG BOND(CBT),USD,"17,421,000",8.7%,5.8%,3.5%,1.7%
5,WN,US ULTRA BOND CBT,USD,"18,076,000",9.0%,6.0%,3.6%,1.8%
6,IK,Euro-BTP Future,EUR,"19,046,000",9.5%,6.3%,3.8%,1.9%
7,OAT,Euro-OAT Future,EUR,"19,878,000",9.9%,6.6%,4.0%,2.0%
8,RX,EURO-BUND FUTURE,EUR,"20,831,000",10.4%,6.9%,4.2%,2.1%
9,TU,US 2YR NOTE (CBT),USD,"31,136,000",15.6%,10.4%,6.2%,3.1%


In [178]:
def BDP(L,YK):
    T = [L[i] + 'A ' + YK[i] for i in range(len(L))]
    T1 = [L[i] + '1 ' + YK[i] for i in range(len(L))]
    
    BDP = blp.bdp(tickers=T, flds=['name','currency','country_iso','exch_code','fut_ctd_isin']).loc[T,:]
    isin = BDP.loc[T,['fut_ctd_isin']].dropna(how='all')
    isin = [i for i in isin['fut_ctd_isin']]

    tmp0 = ['year','dur','yld']
    tmp1 = ['mty_years','dur_adj_bid','yld_ytm_bid']


    BDP1 = blp.bdp(tickers = [i+' Corp' for i in isin], flds=tmp1)
    
    for i in isin:
        for j in range(len(tmp0)):
            BDP.loc[BDP['fut_ctd_isin']==i,tmp0[j]] = BDP1.loc[i+' Corp',tmp1[j]]

    BDP.insert(0,'ticker', L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=['contract_value','volume_avg_20d','volatility_20d']).loc[T1,:]
    BDP1.columns = ['contract_value','volume','vola']
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)
    

    BDP = BDP.rename(columns={'currency':'crncy','country_iso':'exch','exch_code':'exch_code','undl_spot_ticker':'undl'})
    
    for j in ['contract_value','volume']:
        BDP[j] = BDP[j].apply('{:,.0f}'.format)
        
    for j in ['year','dur','yld','vola']:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(blp.bdh(T1[i], 'px_last', '19991201', '20210101', Per='Q').reset_index().iloc[0,0]).strftime('%Y%m')]
    BDP['initinal_t/d'] = tmp
    BDP = BDP.replace('199912', '-')
    
    return BDP
    
L = ['TU','FV','TY','US','WN','DU','OE','RX','UB','OAT','IK','G ','CN','XM','JB']
# L = ['TU','FV']
YK = ['Comdty'] * len(L)

print('Bond Futures')
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Bond Futures
(15, 13)


,ticker,name,crncy,exch,exch_code,fut_ctd_isin,year,dur,yld,contract_value,volume,vola,initinal_t/d
0,TU,US 2YR NOTE (CBT) Jun25,USD,US,CBT,US91282CKE02,1.96,1.87,4.04,"206,758","876,200",1.49,-
1,FV,US 5YR NOTE (CBT) Jun25,USD,US,CBT,US91282CLK52,4.43,4.03,4.08,"107,625","1,513,414",3.30,-
2,TY,US 10YR NOTE (CBT)Jun25,USD,US,CBT,US91282CMK44,6.84,5.83,4.23,"110,406","2,369,027",4.62,-
3,US,US LONG BOND(CBT) Jun25,USD,US,CBT,US912810QN19,15.89,11.05,4.64,"115,656","494,695",8.20,-
4,WN,US ULTRA BOND CBT Jun25,USD,US,CBT,US912810TL26,27.64,15.58,4.78,"120,062","348,418",10.35,201003
5,DU,EURO-SCHATZ FUT Jun25,EUR,DE,EUX,DE000BU22080,1.94,1.88,2.08,"106,865","867,470",2.07,-
6,OE,EURO-BOBL FUTURE Jun25,EUR,DE,EUX,DE000BU25042,5.05,4.69,2.38,"117,550","1,004,618",6.55,-
7,RX,EURO-BUND FUTURE Jun25,EUR,DE,EUX,DE000BU2Z023,8.88,7.91,2.70,"128,470","1,446,422",11.06,-
8,UB,EURO-BUXL 30Y BND Jun25,EUR,DE,EUX,DE000BU2D004,29.37,19.30,3.12,"118,360","163,020",22.85,-
9,OAT,Euro-OAT Future Jun25,EUR,DE,EUX,FR001400QMF9,9.65,8.14,3.40,"122,620","314,738",10.33,201206


In [179]:
def BDP(L,YK):
    T = [L[i] + 'A ' + YK[i] for i in range(len(L))]
    T1 = [L[i] + '1 ' + YK[i] for i in range(len(L))]

    BDP = blp.bdp(tickers=T, flds=['name','currency','country_iso','exch_code','undl_spot_ticker']).loc[T,:]
    BDP.insert(0,'ticker', L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=['contract_value','volume_avg_20d','volatility_20d']).loc[T1,:]
    BDP1.columns = ['contract_value','volume','vola']
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)

    BDP = BDP.rename(columns={'currency':'crncy','country_iso':'exch','exch_code':'exch_code','undl_spot_ticker':'undl'})
    
    tmp = ['contract_value','volume']
    for j in tmp:
        BDP[j] = BDP[j].apply('{:,.0f}'.format)
        
    for j in ['vola']:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(blp.bdh(T1[i], 'px_last', '19991201', '20220101', Per='Q' ).reset_index().iloc[0,0]).strftime('%Y%m')]
    BDP['initinal_t/d'] = tmp
    BDP = BDP.replace('199912', '-')
    
    return BDP
    
L= ['CL','CO','GC']
YK = ['Comdty'] * len(L)

print('Commodity Futures')
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Commodity Futures
(3, 10)


,ticker,name,crncy,exch,exch_code,undl,contract_value,volume,vola,initinal_t/d
0,CL,WTI CRUDE FUTURE May25,USD,US,NYM,USCRWTIC,"69,410","237,411",14.37,-
1,CO,BRENT CRUDE FUTR May25,USD,GB,ICE,COY,"73,520","283,928",19.97,-
2,GC,GOLD 100 OZ FUTR Jun25,USD,US,CMX,NaN,"307,320","167,910",10.81,-


In [180]:
def BDP(L):
    T = ['CSBS' + i + ' TMUQ Curncy' for i in L]  #Mitsubishi TTM
    tmp = ['px_last','volatility_60d','rsi_30d']
    BDP = blp.bdp(tickers=T, flds=tmp).loc[T,:]
    BDP = BDP.loc[T,:]

    for i in tmp:
        BDP[i.lower()] = round(BDP[i.lower()].astype(float),2)
    BDP.insert(0, 'currency', L )
    BDP.insert(1, 'ticker', T)
    BDP = BDP.reset_index(drop=True)
    BDP = BDP.rename(columns={'volatility_60d':'vola'})

    return BDP

L = ['USD','EUR','GBP','CAD','AUD','NZD','CHF','NOK','SEK','HKD','CNY']

print('FX')
BDP = BDP(L)
print(BDP.shape)
BDP

FX
(11, 5)


,currency,ticker,px_last,vola,rsi_30d
0,USD,CSBSUSD TMUQ Curncy,150.58,8.79,48.54
1,EUR,CSBSEUR TMUQ Curncy,161.92,11.50,52.39
2,GBP,CSBSGBP TMUQ Curncy,194.11,10.43,52.90
3,CAD,CSBSCAD TMUQ Curncy,105.39,10.62,48.75
4,AUD,CSBSAUD TMUQ Curncy,94.70,10.87,47.26
5,NZD,CSBSNZD TMUQ Curncy,86.19,10.88,48.31
6,CHF,CSBSCHF TMUQ Curncy,170.38,8.74,51.71
7,NOK,CSBSNOK TMUQ Curncy,14.26,10.39,60.48
8,SEK,CSBSSEK TMUQ Curncy,14.95,13.06,60.16
9,HKD,CSBSHKD TMUQ Curncy,19.36,8.85,48.52


In [181]:
def BDP(L):
    T = [i + ' Index' for i in L]   
    BDP = blp.bdp(tickers=T, flds=['Security_name','long_comp_name','Currency','volatility_260d']).loc[T,:]
    BDP.insert(0, 'ticker', T)
    BDP = BDP.reset_index(drop=True)
    BDP = BDP.rename(columns={'volatility_260d':'vola'})
    BDP['vola'] = round(BDP['vola'].astype(float),2)

    for i in range(len(BDP)):
        tmp = blp.bdh(BDP.loc[i,'ticker'], ['px_last','tot_return_index_gross_dvds'], '20000101', '20230301')
        
        BDP.loc[i,'PR'] = round(tmp.iloc[len(tmp)-1,0] / tmp.iloc[0,0] - 1,2) 
        BDP.loc[i,'TR'] = round(tmp.iloc[len(tmp)-1,1] / tmp.iloc[0,1] - 1,2) 

        if tmp.iloc[len(tmp)-1,1] - tmp.iloc[len(tmp)-1,0] == 0:
            BDP.loc[i,'PR=TR'] = '1'
        else:
            BDP.loc[i,'PR=TR'] = ''

        BDP.loc[i,'Initial_TD'] = tmp.index[0].strftime('%Y%m')
    BDP = BDP.replace('200001', '-')
    return BDP
# https://www.jpmorgan.com/insights/research/index-research
# https://www.jpmorgan.com/content/dam/jpm/cib/complex/content/markets/index-research/Global-Index-Research-Product-Guide-2022.pdf#

In [182]:
print('Japan Stocks')
L = ['TPX','TPXDDVD']
BDP(L)

Japan Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,TPX Index,Tokyo Stock Exchange Tokyo Sto,Tokyo Stock Exchange Tokyo Stock Price Index T...,JPY,23.93,0.16,0.75,,-
1,TPXDDVD Index,Topix Total Return Index JPY,Topix Total Return Index JPY,JPY,23.94,0.75,0.75,1,-


In [183]:
print('Developed Country Stocks')
L = ['MBKO','M0KO','MXKO','M2KO','MAKO','MSHJKOK','GJ124280','HJ124336']
BDP(L)

Developed Country Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,MBKO Index,MSCI Kokusai JPY Price Index,MSCI Kokusai JPY Price Index,JPY,17.70,1.77,1.77,1,-
1,M0KO Index,MSCI Kokusai Gross Total Retur,MSCI Kokusai Gross Total Return JPY Index,JPY,17.70,3.77,3.77,1,-
2,MXKO Index,MSCI KOKUSAI Index,MSCI KOKUSAI Index,USD,12.31,1.06,2.63,,-
3,M2KO Index,MSCI Kokusai Gross Total Retur,MSCI Kokusai Gross Total Return USD Index,USD,12.32,2.59,2.59,1,-
4,MAKO Index,MSCI Kokusai JPY Net Index,MSCI Kokusai JPY Net Index,JPY,17.70,3.28,3.28,1,-
5,MSHJKOK Index,MSCI Hedged Indices In JPY Kok,MSCI Hedged Indices In JPY Kokusai,JPY,NaN,0.44,0.44,1,-
6,GJ124280 Index,MSCI Kokusai 100% Hedged to JP,MSCI Kokusai 100% Hedged to JPY Index,JPY,12.04,1.50,1.50,1,-
7,HJ124336 Index,MSCI Kokusai 100% Hedged to JP,MSCI Kokusai 100% Hedged to JPY,JPY,12.03,-0.00,-0.00,1,202102


In [184]:
print('Emerging Markets Stocks')
L = ['GDUEEGF','NDUEEGF','HJ137561']
BDP(L)

Emerging Markets Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,GDUEEGF Index,MSCI Daily TR Gross EM USD,MSCI Daily TR Gross EM USD,USD,13.86,2.68,2.68,1,-
1,NDUEEGF Index,MSCI Emerging Net Total Return,MSCI Emerging Net Total Return USD Index,USD,13.85,2.41,2.41,1,-
2,HJ137561 Index,MSCI EM (Emerging Markets) 100,MSCI EM (Emerging Markets) 100% Hedged to JPY ...,JPY,12.49,1.31,1.31,1,-


In [185]:
print('Developed Country REIT')
L = ['SREITWJJ','SREITJWJ']
BDP(L)

Developed Country REIT


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,SREITWJJ Index,S&P Developed Ex Japan REIT JP,S&P Developed Ex Japan REIT JPY Index,JPY,16.38,2.24,2.24,1,-
1,SREITJWJ Index,S&P Developed Ex Japan REIT To,S&P Developed Ex Japan REIT Total Return JPY I...,JPY,16.39,9.12,9.12,1,-


In [186]:
print('Japan Bonds')
L = ['BPITTO01']
BDP(L)

Japan Bonds


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,BPITTO01 Index,BPI Total Index,BPI Total Index,JPY,3.32,0.37,0.37,1,-


In [187]:
print('Developed Country Bond')
L = ['SBWGNJYC','SBWGNJYU']
BDP(L)

Developed Country Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,SBWGNJYC Index,FTSE Non-JPY WGBI Curr-Hedged,FTSE Non-JPY WGBI Curr-Hedged JPY,JPY,4.00,0.46,0.46,1,-
1,SBWGNJYU Index,FTSE Non-JPY WGBI JPY,FTSE Non-JPY WGBI JPY,JPY,9.13,2.15,2.15,1,-


In [188]:
print('Emerging Market Bond / USD Bond')
L = ['JPGCUJCP','JPGCHJCP','JPEIDIVR','JPGCCOMP']
# L = ['JPGCUJCP','JPGCHJCP','JPEIDIVR','JPEIPLUS','JPEMCOMP','GBIE1545']
tmp = BDP(L)
tmp

Emerging Market Bond / USD Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,JPGCUJCP Index,unhedged Return JPY,J.P. Morgan EMBI Global Diversified unhedged R...,JPY,10.30,1.10,1.10,1,201210
1,JPGCHJCP Index,Hedged JPY,J.P. Morgan EMBI Global Diversified Hedged JPY,JPY,4.55,1.89,1.89,1,-
2,JPEIDIVR Index,JPMorgan Monthly EMBIs,J.P. Morgan EMBI Global Diversified Composite,USD,4.52,3.68,3.68,1,-
3,JPGCCOMP Index,Composite,J.P. Morgan EMBI Global Diversified Composite,USD,4.52,3.68,3.68,1,-


In [189]:
print('Emerging Market Bond / Local Currency Bond')
L = ['JGENDVUJ','JGENDVHJ','JGENGUJG','JGENGHJG','JGENVUJG','JGENVHJG']
tmp = BDP(L)
tmp['Remarks'] = ['GBI EM Div']*2+['GBI EM Global']*2+['GBI EM Global Div']*2
tmp

Emerging Market Bond / Local Currency Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD,Remarks
0,JGENDVUJ Index,Divers,J.P. Morgan GBI-EM Diversified JPY Unhedged,JPY,10.83,1.73,1.73,1,200212,GBI EM Div
1,JGENDVHJ Index,Divers,J.P. Morgan GBI-EM Diversified JPY Hedged,JPY,2.05,0.27,0.27,1,200301,GBI EM Div
2,JGENGUJG Index,JPY,J.P. Morgan GBI-EM Global Composite Unhedged JPY,JPY,10.27,2.57,2.57,1,200112,GBI EM Global
3,JGENGHJG Index,JPY,J.P. Morgan GBI-EM Global Composite Hedged JPY,JPY,1.39,0.53,0.53,1,200112,GBI EM Global
4,JGENVUJG Index,JPY,J.P. Morgan GBI-EM Global Diversified Composit...,JPY,10.72,2.04,2.04,1,200212,GBI EM Global Div
5,JGENVHJG Index,JPY,J.P. Morgan GBI-EM Global Diversified Composit...,JPY,1.80,0.39,0.39,1,200212,GBI EM Global Div
